# Option A - Generate Sphere Manually

In [1]:
out_dir = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HEBSCHER_TMS/derivatives'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [3]:
#Enter coordinates of sphere
x = -30
y = -51
z = -9
radius=5

Generate the Sphere at The Coordinate

In [ ]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import calculate_voxelwise_mni_coords, calculate_pairwise_distance, generate_sphere_mask, points_within_sphere, save_mask, mask_within_brain
mni_coords_brain = calculate_voxelwise_mni_coords(brain_img)
distances = calculate_pairwise_distance(mni_arr, mni_coords_brain)
points_in_sphere = points_within_sphere(distances, radius)
sphere_mask = generate_sphere_mask(points_in_sphere, brain_img)
sphere_mask = mask_within_brain(sphere_mask, brain_img)
img = save_mask(sphere_mask, brain_img, mni_arr, out_dir)
img

In [ ]:
import nilearn
nilearn.datasets.fetch_atlas_talairach('ba', data_dir='/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/talairach_spaces', verbose=1)

# Option B - Generate Spheres from a CSV

In [1]:
out_dir = '/Volumes/OneTouch/datasets/Kahana_Epilepsy_iEEG/derivatives_unclassified_stim'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [2]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Volumes/OneTouch/datasets/Kahana_Epilepsy_iEEG/metadata/Stim_Effect_Data_with_Session_Column.csv'
sheet = None #'master_list_proper_subjects'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df


# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [5]:
drop_list = ['Subject_ID', 'Cathode']

In [6]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)

Create x,y,z columns if not present

In [7]:
coord_col = 'Cathode_Coords_MNI'

In [ ]:
# import numpy as np 
# import pandas as pd
# try:
#     data_df[coord_col] = data_df[coord_col].str.replace(',', '', regex=False)
#     data_df[coord_col] = data_df[coord_col].apply(lambda x: np.array(x.split(), dtype=float))
# except:
#     print('column already prepared')
# data_df

In [ ]:
# # Specify the column containing the coordinates
# coord_col = 'Cathode_Coords_MNI'

# # Split the column into x, y, z values while skipping invalid rows
# def extract_coords(row):
#     try:
#         # Attempt to split and convert to floats
#         coords = [float(value) for value in str(row).replace(',', ' ').split()]
#         if len(coords) == 3:  # Ensure exactly 3 values
#             return coords
#     except ValueError:
#         pass
#     # Return None for invalid rows
#     return [None, None, None]

# # Apply the function and create the new columns
# data_df[['x_mni_cathode', 'y_mni_cathode', 'z_mni_cathode']] = pd.DataFrame(
#     data_df[coord_col].apply(extract_coords).tolist(),
#     index=data_df.index
# )

# data_df

Optional - Convert Coordinates to Different Space

In [8]:
# # Perform the conversion
# from calvin_utils.nifti_utils.convert_coordinates import convert_coordinates_in_df
# df_converted = convert_coordinates_in_df(data_df, 'x_tal', 'y_tal', 'z_tal', 'talairach_to_mni')
# display(df_converted)

Save changes to CSV if needed

In [9]:
# data_df.to_csv(input_csv_path.split('.csv')[0] + '_updated_coords.csv')

Generate spheres given x, y, and z columns

In [ ]:
data_df

In [11]:
radius = 2
xcol = 'x_mni_cathode'
ycol = 'y_mni_cathode'
zcol = 'z_mni_cathode'
subject_col = 'Subject_ID'
session_col = 'session'

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import generate_spherical_rois_from_df
generate_spherical_rois_from_df(data_df, xcol, ycol, zcol, subject_col, radius, mask_path, out_dir, session_col=session_col, project_on_to_brain=True)

Done
- Calvin